In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Create a dropdown widget for environment selection
interface_dropdown = widgets.Dropdown(
    options=["LAWSON-0005", "LAWSON-0006", "LAWSON-0007", "LAWSON-0008"],
    value="LAWSON-0006",
    description="Interface:",
)

# Display the widget
# display(widgets.HBox([interface_dropdown, outputdirectory]))
display(widgets.HBox([interface_dropdown]))

### Read Configuration Data

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
# Database connection parameters
db_user = 'root'
db_password = 'strong_password'
db_host = 'localhost'
db_name = 'yourdb'
db_port = 33062
interface_id = interface_dropdown.value

# Create a database connection
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
print(interface_id)

In [ ]:
# Query to read data for a specific interfaceID
queryInterface = f"SELECT * FROM interface WHERE interface_id = '{interface_id}'"
# Load data into dataframe
dfInterface = pd.read_sql(queryInterface, engine)
display(dfInterface)

In [ ]:
# Query to read data for a specific interfaceID
queryInterface = f"SELECT * FROM interface WHERE interface_id = '{interface_id}'"

print(queryInterface)



# Load data into dataframe
dfInterface = pd.read_sql(queryInterface, engine)

mappingId = dfInterface['mapping_id'].astype(str).to_string(index=False)

# Query to read data for a specific interfaceID
queryMapping = f"SELECT * FROM mapping WHERE mapping_id = '{mappingId}'"

# Load data into dataframe
dfMapping = pd.read_sql(queryMapping, engine)

display(dfInterface)
display(dfMapping)

#### Read Fixed Length File

In [ ]:
#delimiter = dfMapping['source_delimiter'].to_string(index=False)
print(dfInterface)

In [ ]:
import pandas as pd


#LOGIC FOR FIXED WIDTH FILE
if(dfInterface['source_type'].head(1).to_string(index=False).upper()== 'FIXED'):
    colspecs = [(row['start_position'], row['start_position'] + row['source_length']) for index, row in dfMapping.iterrows() if row['source_field'] is not None]
    names = [row['source_field'] for index, row in dfMapping.iterrows() if row['source_field'] is not None]  # Column names
    dfSource = pd.read_fwf('SourceFileFixedWidth.dat', colspecs=colspecs, names=names, header=None)

#LOGIC FOR DELIMITED FILE
elif(dfInterface['source_type'].to_string(index=False).upper() == 'DELIMITED'):
    delimiter = dfInterface['source_delimiter'].to_string(index=False)
    print(delimiter)
    names = [row['source_field'] for index, row in dfMapping.iterrows() if row['source_field'] is not None]  # Column names
    sourcefile = dfInterface['source_path'].to_string(index=False)
    dfSource = pd.read_csv('SampleFiles/05.TXT', delimiter=delimiter, names=names, header=None)
    # column_types = {'col1': int, 'col2': str, 'col3': str, 'col3': float}
# dfSource3 = pd.read_csv('Delimited.csv', delimiter=',', header=None, dtype=column_types)
    
#dfSource = pd.read_csv('SourceFileDelimited.dat', delimiter='|', names=names, header=None)

print(dfSource)

#### Map to Destination

In [ ]:
# Create an empty DataFrame with the same columns as dfSource
dfDestination = pd.DataFrame(columns=dfSource.columns)
#Solidify the Mapping logic and data transformation
    

destinationFields = ''
for index, row in dfMapping.iterrows():
    destination = row['destination_field']
    source = row['source_field']
    mapping_function = row['mapping_function']
    destination_type = row['destination_data_type']
    if mapping_function == 'PASSTHROUGH':
        if index == len(dfMapping) - 1:
            destinationFields += destination
        else:
            destinationFields += destination + ','
    if mapping_function is None:
        if index == len(dfMapping) - 1:
            destinationFields += f'{source} AS {destination}'
        else:
            destinationFields += f'{source} AS {destination},'
    else:
        if index == len(dfMapping) - 1:
            destinationFields += f'{mapping_function} AS {destination}'
        else:
            destinationFields += f'{mapping_function} AS {destination},'

destinationQuery = f"SELECT {destinationFields} FROM dfSource"
print(destinationQuery)
print(destinationFields)

In [ ]:
import pandasql as psql

dfDestination = psql.sqldf(destinationQuery, globals())

print(dfDestination)

#### Load into Destination

In [ ]:
# Extract the destination table name from the interface table
destination_table = dfInterface['destination_type'].iloc[0]
print(destination_table)
# Load the dataframe into the SQL table
dfDestination.to_sql(destination_table, engine, if_exists='replace', index=False)

print(f"Data loaded into table {destination_table} successfully.")